In [ ]:
## DEFINE VARIABLES ##
hs_url = "http://hydroportal.cuahsi.org/para_la_naturaleza/cuahsi_1_1.asmx?WSDL"
hs_name = "Para la Naturaleza"

In [134]:
!pip install xmltodict
!pip install pandas
!pip install numpy
!pip install owslib
!pip install suds-py3
!pip install folium
!pip install plotly

In [137]:
##IMPORT THE DEPENDENCIES
import xmltodict
import json
import calendar
import pandas as pd
import numpy as np
import folium
import plotly.graph_objects as go
import xml.etree.ElementTree as ET
from owslib.waterml.wml11 import WaterML_1_1 as wml11
from suds.client import Client  # For parsing WaterML/XML
from json import dumps, loads
from datetime import datetime

In [71]:
def parseJSON(json):
    hs_sites = []
    sites_object = None
    # This is to handle the WMO la Plata endpoints ##

    if "Site" in json:
        sites_object = json['Site']
        # If statement is executed for multiple sites within the HydroServer, if there is a single site then it goes to the else statement
        # Parse through the HydroServer and each site with its metadata as a
        # dictionary object to the hs_sites list
        if type(sites_object) is list:
            for site in sites_object:
                hs_json = {}
                latitude = site['Latitude']
                longitude = site['Longitude']
                site_name = site['SiteName']
                site_name = site_name.encode("utf-8")
                network = site["servURL"]
                sitecode = site['SiteCode']

                hs_json["sitename"] = site_name.decode("UTF-8")
                hs_json["latitude"] = latitude
                hs_json["longitude"] = longitude
                hs_json["sitecode"] = sitecode
                hs_json["network"] = network
                hs_json["service"] = "SOAP"
                hs_sites.append(hs_json)
        else:
            hs_json = {}
            latitude = site['Latitude']
            longitude = site['Longitude']
            site_name = site['SiteName']
            site_name = site_name.encode("utf-8")
            network = site["servURL"]
            sitecode = site['SiteCode']

            hs_json["sitename"] = site_name.decode("UTF-8")
            hs_json["latitude"] = latitude
            hs_json["longitude"] = longitude
            hs_json["sitecode"] = sitecode
            hs_json["network"] = network
            hs_json["service"] = "SOAP"
            hs_sites.append(hs_json)

    if "sitesResponse" in json:
        sites_object = json['sitesResponse']['site']

        # If statement is executed for multiple sites within the HydroServer, if there is a single site then it goes to the else statement
        # Parse through the HydroServer and each site with its metadata as a
        # dictionary object to the hs_sites list
        if type(sites_object) is list:
            for site in sites_object:
                hs_json = {}
                latitude = site['siteInfo']['geoLocation'][
                    'geogLocation']['latitude']
                longitude = site['siteInfo']['geoLocation'][
                    'geogLocation']['longitude']
                site_name = site['siteInfo']['siteName']
                site_name = site_name.encode("utf-8")
                network = site['siteInfo']['siteCode']["@network"]
                sitecode = site['siteInfo']['siteCode']["#text"]

                hs_json["sitename"] = site_name.decode("UTF-8")
                hs_json["latitude"] = latitude
                hs_json["longitude"] = longitude
                hs_json["sitecode"] = sitecode
                hs_json["network"] = network
                hs_json["service"] = "SOAP"
                hs_sites.append(hs_json)
        else:
            hs_json = {}
            latitude = sites_object['siteInfo'][
                'geoLocation']['geogLocation']['latitude']
            longitude = sites_object['siteInfo'][
                'geoLocation']['geogLocation']['longitude']
            site_name = sites_object['siteInfo']['siteName']
            site_name = site_name.encode("utf-8")
            network = sites_object['siteInfo']['siteCode']["@network"]
            sitecode = sites_object['siteInfo']['siteCode']["#text"]

            hs_json["sitename"] = site_name.decode("UTF-8")
            hs_json["latitude"] = latitude
            hs_json["longitude"] = longitude
            hs_json["sitecode"] = sitecode
            hs_json["network"] = network
            hs_json["service"] = "SOAP"
            hs_sites.append(hs_json)

    return hs_sites


def add_hs(url):
    return_obj = {}
    client = Client(url, timeout= 500)
    # True Extent is on and necessary if the user is trying to add USGS or
    # Get a list of all the sites and their respective lat lon.
    sites = client.service.GetSites('[:]')
    sites_json={}
    if isinstance(sites, str):
        sites_dict = xmltodict.parse(sites)
        sites_json_object = json.dumps(sites_dict)
        sites_json = json.loads(sites_json_object)
    else:
        sites_json_object = suds_to_json(sites)
        sites_json = json.loads(sites_json_object)

    sites_object = parseJSON(sites_json)
 
    return_obj['siteInfo'] = sites_object
    return return_obj

def get_lats_lngs(objSites):
    sites = objSites['siteInfo']
    sitesLoc = []
    names = []
    for site in sites:
      sitesLoc.append([float(site['latitude']) , float(site['longitude'])])
      names.append(site['sitename'])
    return [sitesLoc,names]


In [72]:
sitesResponse = add_hs(hs_url)
sites = sitesResponse['siteInfo'] ## array containing the sites
print(sites)

##taking the lat, lng, and names from the sites
sitesLocations,sitesNames = get_lats_lngs(sitesResponse)


[{'sitename': 'Río Toro Negro', 'latitude': '18.28559', 'longitude': '-66.4903', 'sitecode': 'Rio_Toro_Negro', 'network': 'Para_La_Naturaleza', 'service': 'SOAP'}, {'sitename': 'Quebrada Batista', 'latitude': '18.19699', 'longitude': '-66.32992', 'sitecode': 'Quebrada_Batista', 'network': 'Para_La_Naturaleza', 'service': 'SOAP'}, {'sitename': 'Río Grande de Manatí', 'latitude': '18.2144', 'longitude': '-66.28665', 'sitecode': 'Rio_Grande_de_Manati', 'network': 'Para_La_Naturaleza', 'service': 'SOAP'}]


In [65]:
# Create a map using Stamen Terrain, centered on Boulder, CO
m = folium.Map(location=[40.0150, -105.2705], 
              tiles = 'Stamen Terrain')

# Add marker for Boulder, CO
for ind in range(len(sitesLocations)):
  folium.Marker(
      location= sitesLocations[ind], # coordinates for the marker (Earth Lab at CU Boulder)
      popup= sitesNames[ind], # pop-up label for the marker
      icon=folium.Icon()
  ).add_to(m)

# Display m
m.fit_bounds(sitesLocations)
m

In [66]:
## Get the variables for the hydroserver
def GetVariables():  
  client = Client(url = hs_url, timeout= 500)

  keywords = client.service.GetVariables('[:]')

  keywords_dict = xmltodict.parse(keywords)
  keywords_dict_object = json.dumps(keywords_dict)

  keywords_json = json.loads(keywords_dict_object)
  array_variables=keywords_json['variablesResponse']['variables']['variable']
  array_keywords_hydroserver=[]

  if isinstance(array_variables,type([])):
      for words in array_variables:
          array_keywords_hydroserver.append(words['variableName'])

  if isinstance(array_variables,dict):
      array_keywords_hydroserver.append(array_variables['variableName'])

  return array_keywords_hydroserver

In [67]:
variables_hs = GetVariables()
print("The variables available in the hydroserver are:")
print(variables_hs)

The variables available in the hydroserver are:
['Water depth, averaged', 'Discharge', 'Velocity']


# **[Get info from a specific site.](https://)**

In [76]:
### If you want a different site just change the name.
chosen_site_name = "Río Toro Negro"
chosen_site_dict = {}

for site in sites:
    if site['sitename'] == chosen_site_name:
        chosen_site_dict = site
        break
else:
    site = None

print(chosen_site_dict)

{'sitename': 'Río Toro Negro', 'latitude': '18.28559', 'longitude': '-66.4903', 'sitecode': 'Rio_Toro_Negro', 'network': 'Para_La_Naturaleza', 'service': 'SOAP'}


In [96]:
## Get Site Info ##
def get_site_info(chosen_site_dict):
    site_code =  chosen_site_dict['sitecode']
    network = chosen_site_dict['network']
    site_desc = network + ':' + site_code

    client = Client(hs_url)

    keywords = client.service.GetVariables('[:]')
    keywords_dict = xmltodict.parse(keywords)
    keywords_dict_object = json.dumps(keywords_dict)

    keywords_json = json.loads(keywords_dict_object)

    site_info_Mc = client.service.GetSiteInfo(site_desc)
    site_info_Mc_dict = xmltodict.parse(site_info_Mc)
    site_info_Mc_json_object = json.dumps(site_info_Mc_dict)
    site_info_Mc_json = json.loads(site_info_Mc_json_object)

    object_methods= site_info_Mc_json['sitesResponse']['site']['seriesCatalog']['series']

    object_with_methods_and_variables = {}
    object_with_descriptions_and_variables = {}
    object_with_time_and_variables = {}
    if(isinstance(object_methods,(dict))):
        variable_name_ = object_methods['variable']['variableName']
        if 'method' in object_methods:
            object_with_methods_and_variables[variable_name_]= object_methods['method']['@methodID']
        else:
            object_with_methods_and_variables[variable_name_]= None
        object_with_descriptions_and_variables[variable_name_]= object_methods['source'];
        object_with_time_and_variables[variable_name_]= object_methods['variableTimeInterval'];
    else:
        for object_method in object_methods:
            variable_name_ = object_method['variable']['variableName']
            if 'method' in object_method:
                object_with_methods_and_variables[variable_name_]= object_method['method']['@methodID']
            else:
                object_with_methods_and_variables[variable_name_]= None
            object_with_descriptions_and_variables[variable_name_]= object_method['source'];
            object_with_time_and_variables[variable_name_]= object_method['variableTimeInterval'];

    array_variables=keywords_json['variablesResponse']['variables']['variable']
    array_keywords_hydroserver=[]
    array_variables_codes = []
    array_variables_lengths = []
    length_values = 0
    if isinstance(array_variables,type([])):
        for words in array_variables:

            variable_text = words['variableName']
            code_variable = words['variableCode']['#text']
            start_date = ""
            end_date = ""
            variable_desc = network + ':' + code_variable
            try:
                values = client.service.GetValues(
                    site_desc, variable_desc, start_date, end_date, "")

                values_dict = xmltodict.parse(values)  # Converting xml to dict
                values_json_object = json.dumps(values_dict)
                values_json = json.loads(values_json_object)
                if 'timeSeriesResponse' in values_json:
                    times_series = values_json['timeSeriesResponse'][
                        'timeSeries']  # Timeseries object for the variable
                    if times_series['values'] is not None:
                        length_values= len(times_series['values']['value'])
                    else:
                        length_values = 0
                else:
                    times_series = values_json['wml1:timeSeriesResponse'][
                        'wml1:timeSeries']  # Timeseries object for the variable
                    if times_series['wml1:values'] is not None:
                        length_values= len(times_series['wml1:values']['wml1:value'])
                    else:
                        length_values = 0

                array_variables_lengths.append(length_values)
                array_keywords_hydroserver.append(words['variableName'])
                array_variables_codes.append(words['variableCode']['#text'])
            except Exception as e:
                print("OOPS",e.__class__)
    if isinstance(array_variables,dict):
        variable_text = array_variables['variableName']
        code_variable = array_variables['variableCode']['#text']
        start_date = ""
        end_date = ""
        variable_desc = network + ':' + code_variable
        try:
            values = client.service.GetValues(
                site_desc, variable_desc, start_date, end_date, "")

            values_dict = xmltodict.parse(values)  # Converting xml to dict
            values_json_object = json.dumps(values_dict)
            values_json = json.loads(values_json_object)
            if 'timeSeriesResponse' in values_json:
                times_series = values_json['timeSeriesResponse'][
                    'timeSeries']  # Timeseries object for the variable
                if times_series['values'] is not None:
                    length_values= len(times_series['values']['value'])
                else:
                    length_values = 0
            else:
                times_series = values_json['wml1:timeSeriesResponse'][
                    'wml1:timeSeries']  # Timeseries object for the variable
                if times_series['wml1:values'] is not None:
                    length_values= len(times_series['wml1:values']['wml1:value'])
                else:
                    length_values = 0


            array_variables_lengths.append(length_values)

            array_keywords_hydroserver.append(array_variables['variableName'])
            array_variables_codes.append(array_variables['variableCode']['#text'])
        except Exception as e:
            print("OOPS",e.__class__)
    return_obj={}
    return_obj['variables_names']=array_keywords_hydroserver
    return_obj['variables_codes']=array_variables_codes
    return_obj['variables_counts'] = array_variables_lengths
    return_obj['variables_methodsIDs']= object_with_methods_and_variables
    return_obj['variables_description'] = object_with_descriptions_and_variables
    return_obj['variables_times_series_description'] = object_with_time_and_variables
    return_obj['fullsiteInfo']= site_info_Mc_json

    return return_obj


In [103]:
info_site = get_site_info(chosen_site_dict)
## The name of the variables in the site
print("The name of the variables in the site \n", info_site['variables_names'])

## The variables codes in the site
print("The variables codes in the site \n" , info_site['variables_codes'])

## The number of values in each variable in the site
print("The number of values in each variable in the site \n", info_site['variables_counts'])

## The method ID of each variable in the site
print("The method ID of each variable in the site \n", info_site['variables_methodsIDs'])

##The description of all the varaibles in the site
print("The method ID of each variable in the site \n", info_site['variables_description'])

##The description of all the time series of each variable in the site
print("The description of all the time series of each variable in the site \n", info_site['variables_times_series_description'])

##The fullSite Information about the site

print("The fullSite Information about the site \n",info_site['siteInfo'])





The name of the variables in the site 
 ['Water depth, averaged', 'Discharge', 'Velocity']
The variables codes in the site 
 ['Average_Stream_Depth', 'Total_Flow', 'Average_Stream_Velocity']
The number of values in each variable in the site 
 [21, 21, 21]
The method ID of each variable in the site 
 {'Water depth, averaged': '1', 'Discharge': '2', 'Velocity': '3'}
The method ID of each variable in the site 
 {'Water depth, averaged': {'@sourceID': '1', 'organization': 'Para La Naturaleza', 'sourceDescription': 'Para La Naturaleza and National Science Foundation (Grant No. 1223882) sponsored this Citizen Science project about the hydrology of three streams in the Rio Grande de Manatí Watershed in Puerto Rico.', 'citation': 'Para La Naturaleza NSF'}, 'Discharge': {'@sourceID': '1', 'organization': 'Para La Naturaleza', 'sourceDescription': 'Para La Naturaleza and National Science Foundation (Grant No. 1223882) sponsored this Citizen Science project about the hydrology of three streams in

# Get Time series from a specific variable

In [112]:
## just change this for any of the variables above ##
chosen_variable_name = 'Water depth, averaged'
chosen_variable_index = info_site['variables_names'].index(chosen_variable_name)
chosen_variable_code = info_site['variables_codes'][chosen_variable_index]
chosen_variable_methodID = info_site['variables_methodsIDs'][chosen_variable_name]

##now we will get the time frame for the time series of the variable, you can change it to any value in within later ##
variabletimeSeriesInfo = info_site['variables_times_series_description'][chosen_variable_name]
beginningTime= variabletimeSeriesInfo['beginDateTime'].split('T')[0]
endingTime= variabletimeSeriesInfo['endDateTime'].split('T')[0]

##create a dict for the variable values##
chosen_variable_dict = {}
chosen_variable_dict['code'] = chosen_variable_code
chosen_variable_dict['startDate'] = beginningTime
chosen_variable_dict['endDate'] = endingTime
chosen_variable_dict['methodID'] = chosen_variable_methodID


In [125]:
def get_values_graph_hs(chosen_site_dict,chosen_variable_dict):
    return_obj={}
    site_code =  chosen_site_dict['sitecode']
    network = chosen_site_dict['network']
    site_desc = network + ':' + site_code
    code_variable = chosen_variable_dict['code']
    start_date = chosen_variable_dict['startDate']
    end_date = chosen_variable_dict['endDate']
    actual_methodsID = chosen_variable_dict['methodID']

    variable_desc = network + ':' + code_variable

    client = Client(hs_url)  # Connect to the HydroServer endpoint


    values = client.service.GetValues(
        site_desc, variable_desc, start_date, end_date, "")
    values_dict = xmltodict.parse(values)  # Converting xml to dict
    values_json_object = json.dumps(values_dict)
    values_json = json.loads(values_json_object)
    times_series = {}
    if 'timeSeriesResponse' in values_json:

        times_series = values_json['timeSeriesResponse'][
            'timeSeries']  # Timeseries object for the variable
        if times_series['values'] is not None:

            graph_json = {}  # json object that will be returned to the front end
            graph_json["variable"] = times_series['variable']['variableName']
            graph_json["unit"]=""
            if times_series['variable']['unit']['unitAbbreviation'] is not None:
                graph_json["unit"] = times_series[
                    'variable']['unit']['unitAbbreviation']

            graph_json["title"] = times_series['variable']['variableName'] + " (" + graph_json["unit"] + ") vs Time"
            for j in times_series['values']:  # Parsing the timeseries
                data_values = []
                data_values2 = []
                if j == "value":
                    if type(times_series['values']['value']) is list:

                        count = 0
                        for k in times_series['values']['value']:
                            return_obj['k']= k

                            try:
                                if k['@methodCode'] == actual_methodsID:
                                    count = count + 1
                                    time = k['@dateTimeUTC']
                                    time1 = time.replace("T", "-")
                                    time_split = time1.split("-")
                                    year = int(time_split[0])
                                    month = int(time_split[1])
                                    day = int(time_split[2])
                                    hour_minute = time_split[3].split(":")
                                    hour = int(hour_minute[0])
                                    minute = int(hour_minute[1])
                                    value = float(str(k['#text']))
                                    date_string = datetime(
                                        year, month, day, hour, minute)

                                    date_string_converted = date_string.strftime("%Y-%m-%d %H:%M:%S")
                                    data_values2.append([date_string_converted,value])
                                    data_values2.sort()
                                    time_stamp = calendar.timegm(
                                        date_string.utctimetuple()) * 1000
                                    data_values.append([time_stamp, value])
                                    data_values.sort()
                                graph_json["values2"] = data_values2
                                graph_json["count"] = count
                            except KeyError:  # The Key Error kicks in when there is only one timeseries
                                count = count + 1
                                time = k['@dateTimeUTC']
                                time1 = time.replace("T", "-")
                                time_split = time1.split("-")
                                year = int(time_split[0])
                                month = int(time_split[1])
                                day = int(time_split[2])
                                hour_minute = time_split[3].split(":")
                                hour = int(hour_minute[0])
                                minute = int(hour_minute[1])
                                value = float(str(k['#text']))
                                date_string = datetime(
                                    year, month, day, hour, minute)
                                data_values2.append([date_string,value])
                                data_values2.sort()
                                time_stamp = calendar.timegm(
                                    date_string.utctimetuple()) * 1000
                                data_values.append([time_stamp, value])
                                data_values.sort()
                            graph_json["values"] = data_values2
                            graph_json["count"] = count
                            return_obj['graphs']=graph_json

                    else:  # The else statement is executed is there is only one value in the timeseries
                        try:
                            if times_series['values']['value']['@methodCode'] == actual_methodsID:
                                time = times_series['values'][
                                    'value']['@dateTimeUTC']
                                time1 = time.replace("T", "-")
                                time_split = time1.split("-")
                                year = int(time_split[0])
                                month = int(time_split[1])
                                day = int(time_split[2])
                                hour_minute = time_split[3].split(":")
                                hour = int(hour_minute[0])
                                minute = int(hour_minute[1])
                                value = float(
                                    str(times_series['values']['value']['#text']))

                                date_string = datetime(
                                    year, month, day, hour, minute)

                                data_values2.append([date_string,value])
                                data_values2.sort()
                                time_stamp = calendar.timegm(
                                    date_string.utctimetuple()) * 1000
                                data_values.append([time_stamp, value])
                                data_values.sort()
                                graph_json["values2"] = data_values2

                                # graph_json["values"] = data_values
                                graph_json["count"] = 1
                                return_obj['graphs']=graph_json

                        except KeyError:
                            time = times_series['values'][
                                'value']['@dateTimeUTC']
                            time1 = time.replace("T", "-")
                            time_split = time1.split("-")
                            year = int(time_split[0])
                            month = int(time_split[1])
                            day = int(time_split[2])
                            hour_minute = time_split[3].split(":")
                            hour = int(hour_minute[0])
                            minute = int(hour_minute[1])
                            value = float(
                                str(times_series['values']['value']['#text']))
                            date_string = datetime(
                                year, month, day, hour, minute)

                            time_stamp = calendar.timegm(
                                date_string.utctimetuple()) * 1000
                            data_values.append([time_stamp, value])
                            data_values.sort()

                            data_values2.append([date_string,value])
                            data_values2.sort()
                            graph_json["values"] = data_values2
                            graph_json["count"] = 1
                            return_obj['graphs']=graph_json

    else:
        times_series = values_json['wml1:timeSeriesResponse'][
            'wml1:timeSeries']  # Timeseries object for the variable

        return_obj['values'] = times_series
        if times_series['wml1:values'] is not None:


            return_obj['values'] = times_series

            graph_json = {}  # json object that will be returned to the front end
            graph_json["variable"] = times_series['wml1:variable']['wml1:variableName']
            graph_json["unit"]=""
            if times_series['wml1:variable']['wml1:unit']['wml1:unitAbbreviation'] is not None:
                graph_json["unit"] = times_series[
                    'wml1:variable']['wml1:unit']['wml1:unitAbbreviation']


            graph_json["title"] = times_series['wml1:variable']['wml1:variableName'] + " (" + graph_json["unit"] + ") vs Time"
            for j in times_series['wml1:values']:  # Parsing the timeseries
                data_values = []
                data_values2 = []
                if j == "wml1:value":
                    if type(times_series['wml1:values']['wml1:value']) is list:
                        count = 0
                        for k in times_series['wml1:values']['wml1:value']:
                            return_obj['k']= k

                            try:

                                if k['@methodCode'] == actual_methodsID:
                                    count = count + 1
                                    time = k['@dateTimeUTC']
                                    time1 = time.replace("T", "-")


                                    time_split = time1.split("-")
                                    year = int(time_split[0])
                                    month = int(time_split[1])
                                    day = int(time_split[2])
                                    hour_minute = time_split[3].split(":")
                                    hour = int(hour_minute[0])
                                    minute = int(hour_minute[1])
                                    value = float(str(k['#text']))
                                    date_string = datetime(
                                        year, month, day, hour, minute)

                                    date_string_converted = date_string.strftime("%Y-%m-%d %H:%M:%S")
                                    data_values2.append([date_string_converted,value])
                                    data_values2.sort()
                                    time_stamp = calendar.timegm(
                                        date_string.utctimetuple()) * 1000
                                    data_values.append([time_stamp, value])
                                    data_values.sort()
                                graph_json["values2"] = data_values2
                                graph_json["count"] = count
                            except KeyError:  # The Key Error kicks in when there is only one timeseries
                                print("The Key Error kicks in because there was only one timespace")
                                count = count + 1
                                time = k['@dateTimeUTC']
                                time1 = time.replace("T", "-")
                                time_split = time1.split("-")
                                year = int(time_split[0])
                                month = int(time_split[1])
                                day = int(time_split[2])
                                hour_minute = time_split[3].split(":")
                                hour = int(hour_minute[0])
                                minute = int(hour_minute[1])
                                value = float(str(k['#text']))
                                date_string = datetime(
                                    year, month, day, hour, minute)
                                # print(date_string)
                                data_values2.append([date_string,value])
                                data_values2.sort()
                                time_stamp = calendar.timegm(
                                    date_string.utctimetuple()) * 1000
                                data_values.append([time_stamp, value])
                                data_values.sort()
                            graph_json["values"] = data_values2
                            graph_json["count"] = count
                            return_obj['graphs']=graph_json

                    else:  # The else statement is executed is there is only one value in the timeseries
                        try:
                            if times_series['values']['value']['@methodCode'] == actual_methodsID:
                                time = times_series['values'][
                                    'value']['@dateTimeUTC']
                                time1 = time.replace("T", "-")
                                time_split = time1.split("-")
                                year = int(time_split[0])
                                month = int(time_split[1])
                                day = int(time_split[2])
                                hour_minute = time_split[3].split(":")
                                hour = int(hour_minute[0])
                                minute = int(hour_minute[1])
                                value = float(
                                    str(times_series['values']['value']['#text']))

                                date_string = datetime(
                                    year, month, day, hour, minute)
                                data_values2.append([date_string,value])
                                data_values2.sort()
                                time_stamp = calendar.timegm(
                                    date_string.utctimetuple()) * 1000
                                data_values.append([time_stamp, value])
                                data_values.sort()
                                graph_json["values2"] = data_values2
                                graph_json["count"] = 1
                                return_obj['graphs']=graph_json

                        except KeyError:
                            time = times_series['values'][
                                'value']['@dateTimeUTC']
                            time1 = time.replace("T", "-")
                            time_split = time1.split("-")
                            year = int(time_split[0])
                            month = int(time_split[1])
                            day = int(time_split[2])
                            hour_minute = time_split[3].split(":")
                            hour = int(hour_minute[0])
                            minute = int(hour_minute[1])
                            value = float(
                                str(times_series['values']['value']['#text']))
                            date_string = datetime(
                                year, month, day, hour, minute)
                            time_stamp = calendar.timegm(
                                date_string.utctimetuple()) * 1000
                            data_values.append([time_stamp, value])
                            data_values.sort()

                            data_values2.append([date_string,value])
                            data_values2.sort()
                            graph_json["values"] = data_values2
                            graph_json["count"] = 1
                            return_obj['graphs']=graph_json
    ##INTERPOLATION
    ##UNCOMMENT THESE LINES FOR MEAN INTERPOLATION

    # time_pd, values_pd = zip(*graph_json["values2"])
    # pds={}
    # pds['time'] = time_pd
    # pds['value'] = values_pd
    # df_interpolation= pd.DataFrame(pds,columns = ["time","value"])
    # df_interpolation2= pd.DataFrame(pds,columns = ["time","value"])
    # df_interpolation.loc[df_interpolation.value < 0] = np.NaN
    # df_interpolation.replace(0, np.nan, inplace=True)
    # df_interpolation['time'] = pd.to_datetime(df_interpolation['time'])
    # df_interpolation = df_interpolation.set_index('time').resample('D').mean()
    # df_interpolation['value'] = df_interpolation['value'].interpolate()
    # df_interpolation.reset_index(level=0, inplace=True)
    # listVals = df_interpolation['value'].to_list()
    # listTimes = df_interpolation['time'].to_list()
    # dataInterpolated = []
    # for t,v in zip(listTimes,listVals):
    #     dataInterpolated.append([t,v])
    # graph_json['interpolation']=dataInterpolated

    return return_obj

In [144]:
##Get the time series
variable_object = get_values_graph_hs(chosen_site_dict,chosen_variable_dict)['graphs']
timeStamps=[]
valuesTimeSeries = []
for index in variable_object['values']:
  timeStamps.append(index[0])
  valuesTimeSeries.append(index[1])

##PLOT THE TIME SERIES

fig = go.Figure(data=go.Scatter(x=timeStamps, y=valuesTimeSeries))
# Edit the layout
fig.update_layout(title = variable_object ['title'],
                   xaxis_title ='Time',
                   yaxis_title = variable_object ['unit'])
fig.show()
